In [169]:

using ITensors
using ITensors: position!
using KrylovKit
using Printf
using ITensorTDVP

# Simple TDVP

Below is a code for a simple TDVP implementation (both for 1-site & 2-site variant). This is based on the ITensorTDVP. The function tdvp! takes the folllowing arguments.

**Initial wavefunction** <br>
**Hamiltonian :** (in MPO form) to time evolve the wave function<br>
**dt(time step)** <br>
**tf(final time )** : time upto which you want to evolve the system)<br>
**reverse_step_val :** Decides whether you need the right-to-left swap (Changes the accuracy O(dt^3) for true,O(dt^2) for no)

Time is entered as real for time evolution. The i is multiplied internally.

In [157]:
struct TDVP end
set_nsite!(PH::ProjMPO,nsite) = (PH.nsite = nsite)

function solver(H, t, psi0; kwargs...)
    solver_kwargs = (;
      ishermitian=get(kwargs, :ishermitian, true),
      issymmetric=get(kwargs, :issymmetric, true),
      tol=get(kwargs, :solver_tol, 1E-12),
      krylovdim=get(kwargs, :solver_krylovdim, 30),
      maxiter=get(kwargs, :solver_maxiter, 100),
      verbosity=get(kwargs, :solver_outputlevel, 0),
      eager=true,
    )
    psi, info = exponentiate(H, t, psi0; solver_kwargs...)
    return psi, info
end

function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end


function tdvp!(ψ::MPS,H::MPO,dt::Float64,tf::Float64,nsite::Int64,reverse_step_val; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    maxdim::Integer = get(kwargs, :maxdim, typemax(Int))
    mindim::Integer = get(kwargs, :mindim, 1)
    cutoff::Real = get(kwargs, :cutoff, 1E-16)
    
    #If progress bar is used
    #pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
   #Imaginary time step
    τ = 1im*dt
    
    N = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #Projected MPO
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    
    PH = ProjMPO(H)
    position!(PH,ψ,1)
    
    
    
    set_nsite!(PH, nsite)
    
    
    maxtruncerr = 0.0
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        current_time = dt*time_step
        for (b,ha) in sweepnext(N; ncenter=nsite)
                # Do 'forwards' evolution step
                
                
                set_nsite!(PH, nsite)
                #orthogonalise at position b
                
                position!(PH, ψ, b)
                if nsite == 1
                  #println(" Forward phi1 Selected")
                  phi1 = ψ[b]
                elseif nsite == 2
                  phi1 = ψ[b] * ψ[b + 1]
                end

                
                #orthogonalise at position b
                
                #phi1, info = exponentiate(PH, -τ/2, phi1; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                phi1, info = solver(PH, -τ/2, phi1)
                info.converged==0 && throw("exponentiate did not converge")
                
                spec = nothing
                
                
                if nsite == 1
                  ψ[b] = phi1
                  Sz = measure_Sz(ψ,b) 
                  #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
                  flush(stdout)
                  #println(" Forward phi1 Replaced")
                elseif nsite == 2
                  #Determine sweep direction
                  dir = ha == 1 ? "left" : "right"
                  spec = replacebond!(ψ,b,phi1;mindim,cutoff,normalize=normalize, ortho = dir, kwargs... )
                  maxtruncerr = max(maxtruncerr, spec.truncerr)
                end
                
                if(reverse_step_val == true)
                    if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))


                        b1 = (ha==1 ? b + 1 : b)
                        Δ = (ha==1 ? +1 : -1)

                        if nsite == 2
                            phi0 = ψ[b1]
                        elseif nsite == 1
                            #println(" SVD")
                            uinds = uniqueinds(phi1, ψ[b + Δ])
                            U, S, V = svd(phi1, uinds)
                            ψ[b] = U
                            phi0 = S * V
                            if ha == 1
                              ITensors.setleftlim!(ψ, b)
                            elseif ha == 2
                              ITensors.setrightlim!(ψ, b)
                            end
                        end

                        # evolve with single-site Hamiltonian backward in time.
                        # In the case of imaginary time-evolution this step
                        # is not necessary (see Ref. [1])




                        set_nsite!(PH, nsite - 1)
                        ITensors.position!(PH, ψ, b1)
                        phi0, info = exponentiate(PH, τ/2, phi0; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                        normalize && (phi0 ./= norm(phi0))
                        info.converged==0 && throw("exponentiate did not converge")


                        if nsite == 2
                            ψ[b1] = phi0
                            #print(" Reverse phi  calculated")
                        elseif nsite == 1
                            #println(" Reverse phi  calculated")
                            ψ[b + Δ] = phi0 * ψ[b + Δ]
                            if ha == 1
                              ITensors.setrightlim!(ψ, b + Δ + 1)
                            elseif ha == 2
                              ITensors.setleftlim!(ψ, b + Δ - 1)
                            end
                        end

                        set_nsite!(PH, nsite)



                    end
                    #if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))
                end
                #for if(reverse_step_val == true)
        end
        #for(b,ha)
            
            #Sz = measure_Sz(ψ,5) 
            #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
            #flush(stdout)

        
        end
        #time elapsed
        
        
       
        
    end
    #External for loop for time_step
    
    ψ=ψ/norm(ψ)
end
#function


tdvp! (generic function with 1 method)

In [158]:
#Number of spin 1/2 sites
N = 15

#fnal time
tf = 2.0
#time step
tau = 0.01


s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
#ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ=randomMPS(s; linkdims=5)
#psi = productMPS(s, n -> isodd(n) ? "Up" : "Dn")


for j=1:N
    println("site : ",j," ",measure_Sz(ψ,j))
end

ψ4 = deepcopy(ψ)
ψ5 = deepcopy(ψ)

site : 1 0.04368128972185656
site : 2 -0.2996617443564895
site : 3 -0.09959572031063438
site : 4 -0.008983413577377506
site : 5 -0.14797405332140992
site : 6 -0.04375798996271485
site : 7 -0.02098773956131647
site : 8 -0.21278208938971926
site : 9 -0.026386044085882316
site : 10 0.05371836952978594
site : 11 0.046454599439089195
site : 12 0.08163827403479483
site : 13 -0.08495614626104304
site : 14 0.004568633228926915
site : 15 -0.023087169803025907


MPS
[1] ((dim=2|id=304|"S=1/2,Site,n=1"), (dim=5|id=107|"Link,l=1"))
[2] ((dim=5|id=107|"Link,l=1"), (dim=2|id=369|"S=1/2,Site,n=2"), (dim=5|id=895|"Link,l=2"))
[3] ((dim=5|id=895|"Link,l=2"), (dim=2|id=444|"S=1/2,Site,n=3"), (dim=5|id=316|"Link,l=3"))
[4] ((dim=5|id=316|"Link,l=3"), (dim=2|id=134|"S=1/2,Site,n=4"), (dim=5|id=823|"Link,l=4"))
[5] ((dim=5|id=823|"Link,l=4"), (dim=2|id=998|"S=1/2,Site,n=5"), (dim=5|id=479|"Link,l=5"))
[6] ((dim=5|id=479|"Link,l=5"), (dim=2|id=634|"S=1/2,Site,n=6"), (dim=5|id=95|"Link,l=6"))
[7] ((dim=5|id=95|"Link,l=6"), (dim=2|id=772|"S=1/2,Site,n=7"), (dim=5|id=146|"Link,l=7"))
[8] ((dim=5|id=146|"Link,l=7"), (dim=2|id=531|"S=1/2,Site,n=8"), (dim=5|id=567|"Link,l=8"))
[9] ((dim=5|id=567|"Link,l=8"), (dim=2|id=433|"S=1/2,Site,n=9"), (dim=5|id=608|"Link,l=9"))
[10] ((dim=5|id=608|"Link,l=9"), (dim=2|id=363|"S=1/2,Site,n=10"), (dim=5|id=420|"Link,l=10"))
[11] ((dim=5|id=420|"Link,l=10"), (dim=2|id=888|"S=1/2,Site,n=11"), (dim=5|id=501|"Link,l=11"))
[12] (

In [159]:
ψ1 = deepcopy(ψ)
#1-site TDVP
tdvp!(ψ1,H,tau,tf,1,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.
for j=1:N
    println("site : ",j," ",measure_Sz(ψ1,j))
end



site : 1 -0.12186520838536641
site : 2 -0.10747737877670228
site : 3 -0.09631854508018667
site : 4 -0.07631224057035207
site : 5 -0.11309653257711758
site : 6 -0.043958542776634885
site : 7 -0.049817940258400105
site : 8 -0.11457603859340465
site : 9 -0.0893668218615285
site : 10 0.022111963964637434
site : 11 0.05171387669679688
site : 12 0.08597423244356421
site : 13 -0.030287720195226887
site : 14 -0.021749937721700925
site : 15 -0.03308536559424055


In [160]:
ψ2 = deepcopy(ψ)

#2-site TDVP
tdvp!(ψ2,H,tau,tf,2,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ2,j))
end



site : 1 -0.12472815733198064
site : 2 -0.09711222562747984
site : 3 -0.10429632756107682
site : 4 -0.08243108833286344
site : 5 -0.07008844374661334
site : 6 -0.06914980642520122
site : 7 -0.0899591537856311
site : 8 -0.08157910939940896
site : 9 -0.05330228340177728
site : 10 0.012926435815646132
site : 11 0.037201800963558636
site : 12 0.01969330794410301
site : 13 0.012991485200996175
site : 14 -0.022463422615526932
site : 15 -0.02581395636066075


In [161]:
#TEBD
ψ3 = deepcopy(ψ)
cutoff = 1e-15


ttotal = tf


# Make gates (1,2),(2,3),(3,4),...
gates = ITensor[]
for j in 1:(N - 1)
s1 = s[j]
s2 = s[j + 1]
hj =
  op("Sz", s1) * op("Sz", s2) +
  1 / 2 * op("S+", s1) * op("S-", s2) +
  1 / 2 * op("S-", s1) * op("S+", s2)
Gj = exp(-im * tau / 2 * hj)
push!(gates, Gj)
end
# Include gates in reverse order too
# (N,N-1),(N-1,N-2),...
append!(gates, reverse(gates))
c=div(N,2)

for t in 0.0:tau:ttotal
    Sz = expect(ψ3, "Sz"; sites=c)
    #println("$t $Sz")

    t≈ttotal && break

    ψ3 = apply(gates, ψ3; cutoff)
    normalize!(ψ3)
end


#Measure magnetisation at each site
for j=1:N
    println("site : ",j," ",measure_Sz(ψ3,j))
end




site : 1 -0.12472746998474607
site : 2 -0.09711303394660692
site : 3 -0.10429631714523128
site : 4 -0.08243125248312269
site : 5 -0.07008809909475353
site : 6 -0.06914938711449707
site : 7 -0.08995963411739154
site : 8 -0.08157922077903679
site : 9 -0.053302774136669326
site : 10 0.012926937796822682
site : 11 0.03720240779502107
site : 12 0.019693196740163454
site : 13 0.012990809437348455
site : 14 -0.02246363471112338
site : 15 -0.025813472932186998


In [162]:
#1-site TDVP & 2-site TDVP
println(inner(ψ1,ψ2))

0.5616074771663103 + 0.012827060292880856im


In [163]:
#1-site TDVP & TEBD

println(inner(ψ1,ψ3))

0.5616087905888005 + 0.012828874601906935im


In [164]:
#2-site TDVP & TEBD
println(inner(ψ2,ψ3))

0.9999999998722828 + 1.9849070761988885e-6im


In [167]:
println(ψ3)

MPS
[1] ((dim=2|id=304|"S=1/2,Site,n=1"), (dim=2|id=651|"Link,n=1"))
[2] ((dim=2|id=651|"Link,n=1"), (dim=2|id=369|"S=1/2,Site,n=2"), (dim=4|id=37|"Link,n=1"))
[3] ((dim=2|id=444|"S=1/2,Site,n=3"), (dim=8|id=852|"Link,n=1"), (dim=4|id=37|"Link,n=1"))
[4] ((dim=2|id=134|"S=1/2,Site,n=4"), (dim=16|id=960|"Link,n=1"), (dim=8|id=852|"Link,n=1"))
[5] ((dim=2|id=998|"S=1/2,Site,n=5"), (dim=32|id=143|"Link,n=1"), (dim=16|id=960|"Link,n=1"))
[6] ((dim=2|id=634|"S=1/2,Site,n=6"), (dim=64|id=267|"Link,n=1"), (dim=32|id=143|"Link,n=1"))
[7] ((dim=2|id=772|"S=1/2,Site,n=7"), (dim=97|id=715|"Link,n=1"), (dim=64|id=267|"Link,n=1"))
[8] ((dim=2|id=531|"S=1/2,Site,n=8"), (dim=98|id=606|"Link,n=1"), (dim=97|id=715|"Link,n=1"))
[9] ((dim=2|id=433|"S=1/2,Site,n=9"), (dim=64|id=493|"Link,n=1"), (dim=98|id=606|"Link,n=1"))
[10] ((dim=2|id=363|"S=1/2,Site,n=10"), (dim=32|id=879|"Link,n=1"), (dim=64|id=493|"Link,n=1"))
[11] ((dim=2|id=888|"S=1/2,Site,n=11"), (dim=16|id=242|"Link,n=1"), (dim=32|id=879|"Link,n

# TDVP codes from ITensorTDVP.jl

In [176]:
ψ4 = deepcopy(ψ)


ψ4 = tdvp(
  H,
  -0.01*im,
  ψ4;
  nsweeps=200,
  nsite=1,
  reverse_step=true,
  normalize=true,
  maxdim=30,
  cutoff=1e-15,
  outputlevel=0,
) 


for j=1:length(ψ4)
    println("site : ",j," Sz ",measure_Sz(ψ4,j))
end

site : 1 Sz -0.12186521031546105
site : 2 Sz -0.10747737947063883
site : 3 Sz -0.09631854235718655
site : 4 Sz -0.07631223623822611
site : 5 Sz -0.11309652982991449
site : 6 Sz -0.04395853911416635
site : 7 Sz -0.04981793650295997
site : 8 Sz -0.1145760367126679
site : 9 Sz -0.08936681865551482
site : 10 Sz 0.022111965984734284
site : 11 Sz 0.051713879031122234
site : 12 Sz 0.08597423305758674
site : 13 Sz -0.03028771981756148
site : 14 Sz -0.021749938922621732
site : 15 Sz -0.03308536498814564


In [179]:
ψ5 = deepcopy(ψ)


ψ5 = tdvp(
  H,
  -0.01*im,
  ψ5;
  nsweeps=200,
  nsite=2,
  reverse_step=true,
  normalize=true,
  maxdim=30,
  cutoff=1e-15,
  outputlevel=0,
) 


for j=1:length(ψ5)
    println("site : ",j," Sz ",measure_Sz(ψ5,j))
end

site : 1 Sz -0.12472816890903118
site : 2 Sz -0.09711222416104426
site : 3 Sz -0.10429632241331464
site : 4 Sz -0.08243105280016633
site : 5 Sz -0.07008837946947452
site : 6 Sz -0.0691514078708836
site : 7 Sz -0.08995781122679544
site : 8 Sz -0.08157929961662719
site : 9 Sz -0.05330424078001739
site : 10 Sz 0.01293045466705896
site : 11 Sz 0.03719986947260446
site : 12 Sz 0.019693418567325205
site : 13 Sz 0.012991503930719336
site : 14 Sz -0.022463437499704225
site : 15 Sz -0.02581395381280374
